In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from processing_data import process_data
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn

In [2]:
data = pd.read_csv('data_deep_clean_v6.csv')
data.head()

,IDLopHoc,IDSinhVien,MaMonHoc,HocKy,NamHoc,DiemTongKet,QuaMon,DiemTBMon,TenMonHoc,ThuocKCNTT,DotHoc,GioiTinh,MaHoa_XepLoai
0,25933,1677250,3395,HK1,2020-2021,8.9,1,7.49,Logic học,0,2,1,4
1,25933,1677250,3197,HK1,2019-2020,8.0,1,7.52,Kỹ năng xây dựng kế hoạch,0,3,1,3
2,25933,1677250,3203,HK2,2019-2020,7.6,1,7.19,Âm nhạc – Nhạc lý và Guitar căn bản,0,4,1,3
3,25933,1677250,901,HK1,2020-2021,9.5,1,5.87,Cấu trúc rời rạc,1,4,1,5
4,25933,1677250,1076,HK1,2020-2021,7.9,1,7.77,Tương tác người máy,1,5,1,3


In [3]:
lst_sv = data['IDSinhVien'].values
lst_sv = set(lst_sv)
lst_sv = list(lst_sv)
data_Sv = data[data['IDSinhVien'] == lst_sv[0]][['DotHoc','MaMonHoc']]
data_Sv

,DotHoc,MaMonHoc
3260,2,3320
3261,3,3206
3262,4,3245
3263,4,2031
3264,5,3591


In [4]:
def process_data(data):
    # label_subject = LabelEncoder()
    # data['MaMonHoc'] = label_subject.fit_transform(data['MaMonHoc'])
    # print(len(np.unique(data['MaMonHoc'])))
    lst_sv = data['IDSinhVien'].values
    lst_sv = set(lst_sv)
    lst_sv = list(lst_sv)
    X = []
    Y = []
    for sv in lst_sv:
        data_Sv = data[data['IDSinhVien'] == sv][['DotHoc','MaMonHoc','DiemTongKet']]
        data_Sv.head()
        data_Sv = np.array(data_Sv)
        label = data[data['IDSinhVien'] == sv][['MaMonHoc']]
        label = np.array(label)
        i = 0
        while( i < len(data_Sv)-1):
            X_one = np.zeros((15,3))
            
            if data_Sv[i][0] == data_Sv[i+1][0] and data_Sv[i][0] != 2:
                # X.append(X[-1])
                # Y.append(label[i+1])
                i += 1
            elif data_Sv[i][0] == data_Sv[i+1][0] and data_Sv[i][0] == 2:
                i += 1
            else:
                for j in range(i+1):
                    X_one[j] = data_Sv[j]
                X.append(X_one)
                Y.append(label[i+1])
                i += 1
    X = np.array(X)
    y = np.array(Y)
    
    
    return X, y

In [5]:
X, y = process_data(data)

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoder = label_encoder.fit_transform(y)
subject_count = len(np.unique(y_encoder))
y_encoder = to_categorical(y_encoder, num_classes=subject_count)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoder, test_size=0.2, random_state=42)

In [8]:
#Train GRU
model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(GRU(64))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))  # Lớp Dense với hàm kích hoạt ReLU

model.add(Dense(subject_count, activation='softmax'))



In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train,y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
43/43 [==============================] - 5s 33ms/step - loss: 2.9918 - accuracy: 0.1802 - val_loss: 2.9564 - val_accuracy: 0.1837
Epoch 2/50
43/43 [==============================] - 1s 17ms/step - loss: 2.1356 - accuracy: 0.3552 - val_loss: 2.5393 - val_accuracy: 0.3353
Epoch 3/50
43/43 [==============================] - 1s 18ms/step - loss: 1.9285 - accuracy: 0.3851 - val_loss: 2.4274 - val_accuracy: 0.3353
Epoch 4/50
43/43 [==============================] - 1s 19ms/step - loss: 1.8713 - accuracy: 0.3786 - val_loss: 2.2735 - val_accuracy: 0.3353
Epoch 5/50
43/43 [==============================] - 1s 18ms/step - loss: 1.8187 - accuracy: 0.3902 - val_loss: 1.8122 - val_accuracy: 0.3994
Epoch 6/50
43/43 [==============================] - 1s 18ms/step - loss: 1.8174 - accuracy: 0.3997 - val_loss: 1.7541 - val_accuracy: 0.3994
Epoch 7/50
43/43 [==============================] - 1s 17ms/step - loss: 1.7680 - accuracy: 0.4165 - val_loss: 1.8188 - val_accuracy: 0.3790
Epoch 8/50
43

In [11]:
model.save('gru_2_model.h5')

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
y_pred = model.predict(X_test)
print(y_pred.shape)


14/14 [==============================] - 0s 7ms/step
(429, 28)


In [18]:
sorted_indices = np.argsort(y_pred, axis=1)[:, ::-1]

# Lấy chỉ số của 4 giá trị lớn nhất trong mỗi hàng
y_pred = sorted_indices[:, :4]

print("Chỉ số của 4 giá trị lớn nhất cho mỗi hàng:\n", y_pred)

Chỉ số của 4 giá trị lớn nhất cho mỗi hàng:
 [[ 3  6 24  4]
 [17 22 27 19]
 [24 25  3 20]
 ...
 [17 22 27 19]
 [ 3  6 24  4]
 [22 24  2 20]]


In [19]:
a = label_encoder.inverse_transform(y_pred[6])
a

array([3206, 3197, 3196, 3217], dtype=int64)

In [20]:
y_test2 = np.argmax(y_test, axis=1)
t = label_encoder.inverse_transform(y_test2)

In [21]:
dem = 0
for i in range(len(t)):
    if t[i] in label_encoder.inverse_transform(y_pred[i]):
        dem += 1
dem / len(t)

0.8508158508158508

In [22]:
#Train LSTM
model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2]))))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))  

model.add(Dense(subject_count, activation='softmax'))


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train,y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
43/43 [==============================] - 11s 120ms/step - loss: 2.2817 - accuracy: 0.3508 - val_loss: 2.6181 - val_accuracy: 0.3003
Epoch 2/50
43/43 [==============================] - 3s 77ms/step - loss: 1.8587 - accuracy: 0.4063 - val_loss: 2.0890 - val_accuracy: 0.4781
Epoch 3/50
43/43 [==============================] - 3s 77ms/step - loss: 1.7858 - accuracy: 0.4216 - val_loss: 1.8746 - val_accuracy: 0.4781
Epoch 4/50
43/43 [==============================] - 3s 74ms/step - loss: 1.7640 - accuracy: 0.4172 - val_loss: 1.7553 - val_accuracy: 0.4752
Epoch 5/50
43/43 [==============================] - 4s 82ms/step - loss: 1.7600 - accuracy: 0.4230 - val_loss: 1.6884 - val_accuracy: 0.4694
Epoch 6/50
43/43 [==============================] - 3s 76ms/step - loss: 1.7470 - accuracy: 0.4143 - val_loss: 1.7191 - val_accuracy: 0.4781
Epoch 7/50
43/43 [==============================] - 3s 77ms/step - loss: 1.7118 - accuracy: 0.4340 - val_loss: 1.7124 - val_accuracy: 0.3965
Epoch 8/50


In [24]:
model.save('lstm_2_model.h5')


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
y_pred = model.predict(X_test)
print(y_pred.shape)


14/14 [==============================] - 2s 24ms/step
(429, 28)


In [26]:
sorted_indices = np.argsort(y_pred, axis=1)[:, ::-1]

# Lấy chỉ số của 4 giá trị lớn nhất trong mỗi hàng
y_pred = sorted_indices[:, :4]

print("Chỉ số của 4 giá trị lớn nhất cho mỗi hàng:\n", y_pred)

Chỉ số của 4 giá trị lớn nhất cho mỗi hàng:
 [[ 3  6  4 24]
 [17 22 19 27]
 [24 25  3 20]
 ...
 [17 22 19 27]
 [ 3  6  4 24]
 [24 22  2 20]]


In [27]:
y_test2 = np.argmax(y_test, axis=1)
t = label_encoder.inverse_transform(y_test2)

In [28]:
dem = 0
for i in range(len(t)):
    if t[i] in label_encoder.inverse_transform(y_pred[i]):
        dem += 1
dem / len(t)

0.8531468531468531